#### NOTE: This is the pipeline of Activity recognition without any description This is the size of a typical notebook

In [1]:
import sys
%matplotlib inline

from sensiml import SensiML
from sensiml.widgets import *

dsk = SensiML()

In [2]:
dsk.project ='Activity Demo'
dsk.pipeline = "Activity Demo"

### Make a query

In [ ]:
QueryWidget(dsk).create_widget()

### Construct a pipeline

#### Generate Features

In [5]:
dsk.pipeline.reset()
dsk.pipeline.set_input_query('query_activity')

dsk.pipeline.add_transform("Magnitude", params={"input_columns": ['GyroscopeX','GyroscopeY', 'GyroscopeZ']})

dsk.pipeline.add_transform("Windowing", params={"window_size": 300, 
                                                "delta": 300,})

dsk.pipeline.add_transform("MSE Filter", params={"input_column": 'Magnitude_ST_0000', 
                                                "MSE_target": -1.0, 
                                                "MSE_threshold": 0.01})

dsk.pipeline.add_feature_generator(["Mean", 'Standard Deviation','Sum', '25th Percentile'],
                                   function_defaults = {"columns":[u'Magnitude_ST_0000']})

dsk.pipeline.add_transform('Min Max Scale')

features, s = dsk.pipeline.execute()

#### Train the Model

In [16]:
dsk.pipeline.set_validation_method('Stratified K-Fold Cross-Validation', params={'number_of_folds':3,})

dsk.pipeline.set_classifier('PVP', params={"classification_mode":'RBF','distance_mode':'L1'})

dsk.pipeline.set_training_algorithm('Hierarchical Clustering with Neuron Optimization', params = {'number_of_neurons':5})


dsk.pipeline.set_tvo({'validation_seed':2})

model_results, stats = dsk.pipeline.execute()

In [18]:
model.knowledgepack.save('ActivityKP')
model_results.summarize()

TRAINING ALGORITHM: Hierarchical Clustering with Neuron Optimization
VALIDATION METHOD:  Stratified K-Fold Cross-Validation
CLASSIFIER:         PVP

AVERAGE METRICS:
F1-SCORE:    77.9   sigma 13.38
SENSITIVITY: 100.0   sigma 0.00
PRECISION:   69.1   sigma 17.86

--------------------------------------

STRATIFIED K-FOLD CROSS-VALIDATION MODEL RESULTS

MODEL INDEX: Fold 2
ACCURACY: 88.89
NEURONS: 3

MODEL INDEX: Fold 1
ACCURACY: 72.73
NEURONS: 3

MODEL INDEX: Fold 0
ACCURACY: 41.67
NEURONS: 3



### Download Model

In [ ]:
DownloadWidget(dsk).create_widget()